# Predictor basado en CNN
Basado en https://github.com/satojkovic/cnn-text-classification-tf/tree/use_fasttext y https://github.com/jiegzhan/multi-class-text-classification-cnn

## Se importa el texto a predecir

In [26]:
import os
import sys
import json
import logging
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.contrib import learn
from pathlib import Path

import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
from Preprocesador import Preprocesador

logging.getLogger().setLevel(logging.INFO)

carpetaModeloEntrenado = 'trained_model_1528958572'
csvPrueba = 'general-tweets-test1k.csv'

preprocesador = Preprocesador()

#Definir el nombre de las columnas del archivo
cols = ['id','usuario','Texto','Fecha','Idioma','Polaridad']

#Importar el archivo en memoria 
#Se indica que no tiene encabezados
#Se usan los nombres definidos anteriormente los nombres de las columnas
df = pd.read_csv(csvPrueba,header=None, names=cols)

clean_df = df.copy()

#Imprimir las primeras líneas del archivo para validar una carga correcta
df.head()

[':-)', ':-))', '(-:', '((-:', ':)', ':))', '(:', '((:', ':-]', '[-:']
[':-(', ':(', ':-c', ':c', ':-<', ':<', ':-[', ':[', ':-||', '>:[']


,id,usuario,Texto,Fecha,Idioma,Polaridad
0,142532743211917312,carlos__alsina,¿La posición acertada será la Merkel-Sarko-líd...,2011-12-02T10:17:08,es,?
1,142559090676203520,Federicoquevedo,"Ultimo dia de 'cole' esta semana, nada mejor q...",2011-12-02T12:01:50,es,?
2,142559890332856321,Joss682008,Otra de puertas abiertas. Una paisana de Bono ...,2011-12-02T12:05:01,es,?
3,142599953318023170,VicenteVallesTV,"A las 3 en Antena3, ZP deja a Rajoy la aprobac...",2011-12-02T14:44:12,es,?
4,142642819490783233,AlexdelaIglesia,Que frio ☔⚡,2011-12-02T17:34:33,es,?


## Se va a pre-procesar el texto

In [27]:
print ("Pre procesando tweets...\n")
clean_tweet_texts = []
for i in range(df.Texto.count()):
    if( (i+1)%1000 == 0 ):
        print("%d de %d tweets procesados" % ( i+1, df.Texto.count() ))
    clean_tweet_texts.append(preprocesador.tweetCleaner(str(df['Texto'][i])))
print ("Pre procesamiento completado")

clean_df['Texto'] = clean_tweet_texts
clean_df.head()

Pre procesando tweets...

1000 de 1000 tweets procesados
Pre procesamiento completado


,id,usuario,Texto,Fecha,Idioma,Polaridad
0,142532743211917312,carlos__alsina,¿la posición acertada será la merkelsarkolíder...,2011-12-02T10:17:08,es,?
1,142559090676203520,Federicoquevedo,ultimo dia de cole esta semana nada mejor que ...,2011-12-02T12:01:50,es,?
2,142559890332856321,Joss682008,otra de puertas abiertas una paisana de bono l...,2011-12-02T12:05:01,es,?
3,142599953318023170,VicenteVallesTV,a las xnumx en antenaxnumx zp deja a rajoy la ...,2011-12-02T14:44:12,es,?
4,142642819490783233,AlexdelaIglesia,que frio ☔⚡,2011-12-02T17:34:33,es,?


In [29]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [30]:
def predict_unseen_data():
    """Step 0: load trained model and parameters"""
    checkpoint_dir = carpetaModeloEntrenado
    if not checkpoint_dir.endswith('/'):
        checkpoint_dir += '/'
    checkpoint_file = tf.train.latest_checkpoint(checkpoint_dir + 'checkpoints')
    out_dir = os.path.abspath(os.path.join(os.path.curdir, carpetaModeloEntrenado))
    params = json.loads(open(os.path.join(out_dir, 'parametros.json')).read())
    logging.critical('Loaded the trained model: {}'.format(checkpoint_file))

    """Step 1: load data for prediction"""
    test_file = sys.argv[2]
    test_examples = json.loads(open(test_file).read())

    # labels.json was saved during training, and it has to be loaded during prediction
    labels = json.loads(open(os.path.join(out_dir, 'labels.json')).read())
    one_hot = np.zeros((len(labels), len(labels)), int)
    np.fill_diagonal(one_hot, 1)
    label_dict = dict(zip(labels, one_hot))

    x_test = [str(example) for example in clean_df.Texto]
    logging.info('The number of x_test: {}'.format(len(x_test)))

    y_test = None

    vocab_path = os.path.join(checkpoint_dir, "vocab")
    vocab_processor = learn.preprocessing.VocabularyProcessor.restore(vocab_path)
    x_test = np.array(list(vocab_processor.transform(x_test)))
    my_file = Path("fasttext_vocabulario.dat")
    preentrenado = my_file.is_file()
    if preentrenado:
        print('Load pre-trained word vectors')
        embedding = np.load('fasttext_embeddings.npy')

    """Step 2: compute the predictions"""
    graph = tf.Graph()
    with graph.as_default():
        session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
        sess = tf.Session(config=session_conf)

        with sess.as_default():
            saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
            saver.restore(sess, checkpoint_file)

            input_x = graph.get_operation_by_name("input_x").outputs[0]
            dropout_keep_prob = graph.get_operation_by_name("dropout_keep_prob").outputs[0]
            embedding_placeholder = graph.get_operation_by_name('embedding/pre_trained').outputs[0]
            predictions = graph.get_operation_by_name("output/predictions").outputs[0]

            batches = batch_iter(list(x_test), params['batch_size'], 1, shuffle=False)
            all_predictions = []
            for x_test_batch in batches:
                batch_predictions = sess.run(predictions, {input_x: x_test_batch, embedding_placeholder: embedding, dropout_keep_prob: 1.0})
                all_predictions = np.concatenate([all_predictions, batch_predictions])

    if y_test is not None:
        y_test = np.argmax(y_test, axis=1)
        correct_predictions = sum(all_predictions == y_test)

        logging.critical('The accuracy is: {}'.format(correct_predictions / float(len(y_test))))
        logging.critical('The prediction is complete')

    # Save the actual labels back to file
    actual_labels = [labels[int(prediction)] for prediction in all_predictions]
    
    for i in range(df.Polaridad.count()):
        df.Polaridad[i] = actual_labels[i]

In [31]:
predict_unseen_data()
df.head()

CRITICAL:root:Loaded the trained model: D:\respaldo joax\UCR\Maestria computacion\2018-1\NPL\Deep learning\trained_model_1528958572\checkpoints\model-19600
INFO:root:The number of x_test: 1000


Load pre-trained word vectors
INFO:tensorflow:Restoring parameters from D:\respaldo joax\UCR\Maestria computacion\2018-1\NPL\Deep learning\trained_model_1528958572\checkpoints\model-19600


INFO:tensorflow:Restoring parameters from D:\respaldo joax\UCR\Maestria computacion\2018-1\NPL\Deep learning\trained_model_1528958572\checkpoints\model-19600
C:\Users\joax\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,id,usuario,Texto,Fecha,Idioma,Polaridad
0,142532743211917312,carlos__alsina,¿La posición acertada será la Merkel-Sarko-líd...,2011-12-02T10:17:08,es,N
1,142559090676203520,Federicoquevedo,"Ultimo dia de 'cole' esta semana, nada mejor q...",2011-12-02T12:01:50,es,P
2,142559890332856321,Joss682008,Otra de puertas abiertas. Una paisana de Bono ...,2011-12-02T12:05:01,es,N
3,142599953318023170,VicenteVallesTV,"A las 3 en Antena3, ZP deja a Rajoy la aprobac...",2011-12-02T14:44:12,es,N
4,142642819490783233,AlexdelaIglesia,Que frio ☔⚡,2011-12-02T17:34:33,es,NONE


In [32]:
csv = 'tweets_evaluados.csv'
df.to_csv(csv,encoding='utf-8')